In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext lab_black

In [2]:
import sys
import os
import pandas as pd
from os.path import join
import shutil

sys.path.insert(1, "/export/home/cse200093/Jacques_Bio/super_pipe/py_files/utils")
from export_pandas_to_brat import export_pandas_to_brat

RES_DF_DIR = f"/export/home/cse200093/Jacques_Bio/data_bio/super_pipe_res/stats_by_section_on_cim10/{disease}/res_final_filtered.pkl"
RES_BRAT_DIR_NORM = (
    f"/export/home/cse200093/brat_data/Jacques_Bio/brat_{disease}_not_detected_norm"
)
RES_BRAT_DIR = (
    f"/export/home/cse200093/brat_data/Jacques_Bio/brat_{disease}_not_detected"
)
SRC_BRAT_DIR = f"/export/home/cse200093/Jacques_Bio/data_bio/super_pipe_get_stats_by_section_on_cim10/pred/{disease}_pred"

TO_BE_MATCHED = {
    "lupus_erythemateux_dissemine": [
        "Facteur anti-nucléaire",
        "Anti-DNA natif",
        "Anti-Sm",
    ],
    "syndrome_des_anti-phospholipides": [
        "Anti-cardiolipides",
        "anti_B2GP1",
        "anticoagulant_circulant_lupique",
    ],
    "sclerodermie_systemique": ["Anti-RNA pol 3", "Anti-SCL 70"],
}
if not os.path.exists(RES_BRAT_DIR):
    os.makedirs(RES_BRAT_DIR)
if not os.path.exists(RES_BRAT_DIR_NORM):
    os.makedirs(RES_BRAT_DIR_NORM)

NameError: name 'disease' is not defined

In [7]:
summary_df_docs = pd.read_pickle(join(BRAT_DIR, "summary_df_docs.pkl"))

In [12]:
print(
    summary_df_docs[
        summary_df_docs.instance_num == "-1171294118541235901"
    ].observation_blob.iloc[0]
)

             
 
 
1/5 M. MALHERAULT Vincennette 
Cr signé le 05/07/2019 
 
 
 
 
 
HOPITAUX UNIVERSITAIRES 
 
 
HÔPITAL LARIBOISIERE 
2, 18, Rue De Mathieu 
BP 7042 SANTA FE NEW MEXICO 87501USA Cedex 10 
Standard : 01 43 09 62 88 
 
 
 
Pôle IMAGINE 
DÉPARTEMENT DE MÉDECINE 
INTERNE  
Secteur Orange Pavillon Claude Galien 
 
Pr Sidineia HERNANDEZ AMASIFUEN 
Assistante médico-administrative :  
Elena Marie DIRAI 0624210600 
 
UNITE DE MEDECINE INTERNE 
ET IMMUNOLOGIE CLINIQUE 
3ème étage -  0698021403 
 
Responsable : Pr Lalahaharisoa B BATCHEWOUA 
Dr Fawzy KITOKA- PH 
Dr Estera Bianca DASCENCO – CCA 
Dr Estera Bianca ANASTACIO PRIOR - CCA 
 
Assistantes médico-administratives : 
Halimatou WIDSGOWSKI – 01 80 46 85 77 
Lalahaharisoa B GIGNAT – 01 43 43 75 92 
 
Assistantes sociales : 
Ananivi COUSIN RABHI – 06.45.88.31.53 
Berende LOPHANOR – 0751053795 
 
Cadre de santé : 
Mme LANARDOUNE  - 0667464754 
DECT 55226 
 
Avis de Médecine Interne 
 0160681453 DR. 
 
 
 
   
 
 
 
 
 
 
M. MALH

In [18]:
import random

# Load and preprocess res dataframe
# to have a final dataframe with only the
# columns of interest

res_df = pd.read_json(RES_DF_DIR)
res_df["annotation"] = res_df.apply(
    lambda row: "/".join(row["label"]) + "_" + "/".join(row["found"]).replace(" ", ""),
    axis=1,
)
res_df["label"] = "BIO"


def retrieve_span_of_terms_linked_to_measurement(
    term, term_linked_to_measurement, term_span
):
    return [
        term_span[0] + term.find(term_linked_to_measurement),
        term_span[0]
        + term.find(term_linked_to_measurement)
        + len(term_linked_to_measurement),
    ]


res_df["span_converted"] = res_df.apply(
    lambda row: retrieve_span_of_terms_linked_to_measurement(
        row["term"], row["terms_linked_to_measurement"], row["span_converted"]
    ),
    axis=1,
)
res_df["term"] = res_df["terms_linked_to_measurement"]
res_df = res_df[["term", "source", "span_converted", "label", "annotation"]]

for source in res_df["source"].unique():
    shutil.copy(
        join(SRC_BRAT_DIR, source[:-3] + "txt"), join(RES_BRAT_DIR, source[:-3] + "txt")
    )
    export_pandas_to_brat(
        join(RES_BRAT_DIR, source),
        join(RES_BRAT_DIR, source[:-3] + "txt"),
        res_df.loc[res_df["source"] == source],
        "label",
        "span_converted",
        "term",
        "annotation",
    )

In [51]:
import random

# Load and preprocess res dataframe
# to have a final dataframe with only the
# columns of interest
for disease in TO_BE_MATCHED.keys():
    RES_DF_DIR = f"/export/home/cse200093/Jacques_Bio/data_bio/super_pipe_res/stats_by_section_on_cim10/{disease}/res_final_filtered.pkl"
    RES_BRAT_DIR_NORM = (
        f"/export/home/cse200093/brat_data/Jacques_Bio/brat_{disease}_not_detected_norm"
    )
    RES_BRAT_DIR = (
        f"/export/home/cse200093/brat_data/Jacques_Bio/brat_{disease}_not_detected"
    )
    SRC_BRAT_DIR = f"/export/home/cse200093/Jacques_Bio/data_bio/super_pipe_get_stats_by_section_on_cim10/pred/{disease}_pred"
    shutil.rmtree(RES_BRAT_DIR)
    os.makedirs(RES_BRAT_DIR)
    shutil.rmtree(RES_BRAT_DIR_NORM)
    os.makedirs(RES_BRAT_DIR_NORM)
    res_df = pd.read_pickle(RES_DF_DIR)
    res_df["annotation"] = res_df.apply(
        lambda row: "/".join(row["label"])
        + "_"
        + "/".join(row["found"]).replace(" ", ""),
        axis=1,
    )
    res_df["label"] = "BIO"

    def retrieve_span_of_terms_linked_to_measurement(
        term, term_linked_to_measurement, term_span
    ):
        return [
            term_span[0] + term.find(term_linked_to_measurement),
            term_span[0]
            + term.find(term_linked_to_measurement)
            + len(term_linked_to_measurement),
        ]

    res_df["span_converted"] = res_df.apply(
        lambda row: retrieve_span_of_terms_linked_to_measurement(
            row["term"], row["terms_linked_to_measurement"], row["span_converted"]
        ),
        axis=1,
    )
    res_df["term"] = res_df["terms_linked_to_measurement"]
    detected_doc = res_df[res_df.bio.isin(TO_BE_MATCHED[disease])].source.unique()
    not_detected_doc = res_df[~res_df.source.isin(detected_doc)].source.unique()
    res_df = res_df[["term", "source", "span_converted", "label", "annotation"]]
    for file_path in random.sample(list(not_detected_doc), 300):
        source = file_path[:-3] + "ann"
        shutil.copy(
            join(SRC_BRAT_DIR, source[:-3] + "txt"),
            join(RES_BRAT_DIR, source[:-3] + "txt"),
        )
        shutil.copy(
            join(SRC_BRAT_DIR, source[:-3] + "txt"),
            join(RES_BRAT_DIR_NORM, source[:-3] + "txt"),
        )
        shutil.copy(
            join(SRC_BRAT_DIR, source[:-3] + "ann"),
            join(RES_BRAT_DIR, source[:-3] + "ann"),
        )
        if not res_df.loc[res_df["source"] == source].empty:
            export_pandas_to_brat(
                join(RES_BRAT_DIR_NORM, source),
                join(RES_BRAT_DIR_NORM, file_path[:-3] + "txt"),
                res_df.loc[res_df["source"] == source],
                "label",
                "span_converted",
                "term",
                "annotation",
            )
        else:
            open(join(RES_BRAT_DIR_NORM, source), "a").close()

In [3]:
import sys
import os
import pandas as pd
from os.path import join
import shutil

sys.path.insert(1, "/export/home/cse200093/Jacques_Bio/super_pipe/py_files/utils")
from export_pandas_to_brat import export_pandas_to_brat

RES_DF_DIR = (
    f"/export/home/cse200093/Jacques_Bio/data_bio/gold_std_NLP_diabeto/res.json"
)
RES_BRAT_DIR_NORM = (
    f"/export/home/cse200093/brat_data/Jacques_Bio/gold_std_from_NLP_diabeto/norm"
)
SRC_BRAT_DIR = (
    f"/export/home/cse200093/brat_data/Jacques_Bio/gold_std_from_NLP_diabeto/annotation"
)

if not os.path.exists(RES_BRAT_DIR_NORM):
    os.makedirs(RES_BRAT_DIR_NORM)

In [7]:
res_df = pd.read_json(RES_DF_DIR)
res_df["annotation"] = res_df.apply(
    lambda row: "/".join(row["label"]) + "_" + "/".join(row["found"]).replace(" ", ""),
    axis=1,
)
res_df["label"] = "BIO"


def retrieve_span_of_terms_linked_to_measurement(
    term, term_linked_to_measurement, term_span
):
    return [
        term_span[0] + term.find(term_linked_to_measurement),
        term_span[0]
        + term.find(term_linked_to_measurement)
        + len(term_linked_to_measurement),
    ]


res_df["span_converted"] = res_df.apply(
    lambda row: retrieve_span_of_terms_linked_to_measurement(
        row["term"], row["terms_linked_to_measurement"], row["span_converted"]
    ),
    axis=1,
)
res_df["term"] = res_df["terms_linked_to_measurement"]
res_df = res_df[["term", "source", "span_converted", "label", "annotation"]]
for file_path in res_df.source.unique():
    source = file_path[:-3] + "ann"
    shutil.copy(
        join(SRC_BRAT_DIR, source[:-3] + "txt"),
        join(RES_BRAT_DIR_NORM, source[:-3] + "txt"),
    )
    export_pandas_to_brat(
        join(RES_BRAT_DIR_NORM, source),
        join(RES_BRAT_DIR_NORM, file_path[:-3] + "txt"),
        res_df.loc[res_df["source"] == source],
        "label",
        "span_converted",
        "term",
        "annotation",
    )